In [1]:
using Statistics
using Profile

In [2]:
@enum BinInit begin
    LightSpeed  # set limits of initial bin +- 3.1e8 m/s
    MinMaxVel  # set limits of initial bin to min/max velocities of particles in question
end

In [3]:
@enum BinSplit begin
    MiddleSplit  # split bin among middle
    MeanVelSplit  # split bin along mean velocity in bin
end

In [4]:
@enum BinRefinement begin
    Greedy  # refine the first bin that is suitable
    Heaviest  # refine the heaviest bin
    BiggestAvg  # refine bin with largest average weight
end

In [5]:
struct Constants
    k::Float64
end

constants = Constants(1.38064852e-23)

Constants(1.38064852e-23)

In [6]:
mutable struct Particle
    w::Float64
    v::Array{Float64,1}
    octant::Int
end

In [7]:
mutable struct Bin
    w::Float64  # total weight in bin
    nparticles::Int  # total number of particles in bin
    v::Array{Float64,1}  # velocity of bin
    c::Array{Float64,1}  # second moment in bin
    
    v_lo::Array{Float64,1}  # min velocity of bin
    v_hi::Array{Float64,1}  # max velocity of bin
    v_split::Array{Float64,1}  # velocities along which octant splitting is performed
    
    index_start::Int
    index_end::Int # indices of particle array - start, end
    
    level::Int
end

In [8]:
function maxwellian(T, mass)
    return sqrt(constants.k * T / mass) * randn((3,))
end

maxwellian (generic function with 1 method)

In [9]:
function compute_octant!(particle, v_split)
    particle.octant = 1
    if particle.v[1] > v_split[1]
        particle.octant += 1
    end
    if particle.v[2] > v_split[2]
        particle.octant += 2
    end
    if particle.v[3] > v_split[3]
        particle.octant += 4
    end
end

compute_octant! (generic function with 1 method)

In [10]:
function compute_cell!(particle, N_cell, v_lo, v_hi, v_step)
    
    if ((particle.v[1] > v_lo[1]) && (particle.v[1] < v_hi[1]) &&
        (particle.v[2] > v_lo[2]) && (particle.v[2] < v_hi[2]) &&
        (particle.v[3] > v_lo[3]) && (particle.v[3] < v_hi[3]))
        
        i_x = (particle.v[1] - v_lo[1]) / v_step[1]
        i_y = (particle.v[2] - v_lo[2]) / v_step[2]
        i_z = (particle.v[3] - v_lo[3]) / v_step[3]
        
        particle.octant = trunc(Int, i_x) * (N_cell^2) + trunc(Int, i_y) * N_cell + trunc(Int, i_z)
        
        particle.octant += 1
    else
        particle.octant = N_cell^3 + 1
        
        if (particle.v[1] > 0.0)
            particle.octant += 1
        end
        if (particle.v[2] > 0.0)
            particle.octant += 2
        end
        if (particle.v[3] > 0.0)
            particle.octant += 4
        end
    end
end

compute_cell! (generic function with 1 method)

In [11]:
function cell_merge!(particle_list, N_cell, v_lo, v_hi)
    
    bin_list = Array{Bin}(undef, N_cell^3+8)
    v_step = (v_hi - v_lo) / N_cell
#     index_arr = Array{Float64}(undef, 3)
    
    for i in 1: N_cell^3+8
        bin_list[i] = Bin(0.0, 0, [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0], 0, 0, 0)
    end
    
    for particle in particle_list
        compute_cell!(particle, N_cell, v_lo, v_hi, v_step)
        
        bin_list[particle.octant].w += particle.w
        
        bin_list[particle.octant].v += particle.w * particle.v
    end
    
    
    for bin in bin_list
        if (bin.w > 0.0)
            bin.v = bin.v / bin.w
        end
    end
    
    for particle in particle_list
        bin_list[particle.octant].c += particle.w * ((particle.v .- bin_list[particle.octant].v).^2)
    end
                            
    
    for bin in bin_list
        if (bin.w > 0.0)
             bin.c = sqrt.(bin.c / bin.w)
        end
    end
                                    
    return bin_list
end

cell_merge! (generic function with 1 method)

In [12]:
function compute_bin_values!(particle_list, bin)
    bin.w = 0.0
    bin.nparticles = bin.index_end - bin.index_start + 1
    bin.v = [0.0, 0.0, 0.0]
    bin.c = [0.0, 0.0, 0.0]
    
    for i=bin.index_start:bin.index_end
        bin.w += particle_list[i].w
        bin.v += particle_list[i].w .* particle_list[i].v
#         bin.c += particle_list[i].w .* (particle_list[i].v.^2)
    end
    
    if (bin.w > 0.0)
        bin.v = bin.v / bin.w
    end
    
    for i=bin.index_start:bin.index_end
        bin.c += particle_list[i].w * ((particle_list[i].v .- bin.v).^2)
    end
    
    if (bin.w > 0.0)
        bin.c = sqrt.(bin.c / bin.w)
        
    end
end

compute_bin_values! (generic function with 1 method)

In [13]:
function compute_bin_weight_and_split!(particle_list, bin, split_type)
    # when refining, we just need to keep track of the weight and number of particles in a bin
    # also computes the velocities to split the bin along
    
    bin.w = 0.0
    bin.nparticles = bin.index_end - bin.index_start + 1
    bin.v = [0.0, 0.0, 0.0]
    
    for i=bin.index_start:bin.index_end
        bin.w += particle_list[i].w
        bin.v += particle_list[i].w .* particle_list[i].v
    end
    
    if ((split_type == MiddleSplit) || (bin.w <= 0.0))
        bin.v_split = (bin.v_lo + bin.v_hi) / 2
    else
        bin.v_split = bin.v / bin.w
    end
end

compute_bin_weight_and_split! (generic function with 1 method)

In [14]:
function refine_bin!(particle_list, bins_list, bin_id, n_bins, split_type)
    bin_counts = [0, 0, 0, 0, 0, 0, 0, 0]
    
    for particle in particle_list[bins_list[bin_id].index_start:bins_list[bin_id].index_end]
        compute_octant!(particle, bins_list[bin_id].v_split)
    end

    # a better/faster version would operate on lists of pointers to particles, not particle structs themselves
    particles_in_bin = particle_list[bins_list[bin_id].index_start:bins_list[bin_id].index_end]
    
    for particle in particle_list[bins_list[bin_id].index_start:bins_list[bin_id].index_end]
        bin_counts[particle.octant] += 1
    end
    
    bin_counts_copy = bin_counts[:]
    
    for i=2:8
        bin_counts[i] += bin_counts[i-1]
    end
    
    bin_counts_acc_copy = bin_counts[:]
    
    for particle in particle_list[bins_list[bin_id].index_start:bins_list[bin_id].index_end]
        particles_in_bin[bin_counts[particle.octant]] = particle
        bin_counts[particle.octant] -= 1
    end

    particle_list[bins_list[bin_id].index_start:bins_list[bin_id].index_end] = particles_in_bin[:]

    new_bins = 0
    
    for i=1:8
        if bin_counts_copy[i] > 0
            new_bins += 1
            
            
            # set new bin limits, split along z axis
            if i<=4
                vz_min = bins_list[bin_id].v_lo[3]
                vz_max = bins_list[bin_id].v_split[3]
            else
                vz_min = bins_list[bin_id].v_split[3]
                vz_max = bins_list[bin_id].v_hi[3]
            end
                
            
            # set new bin limits, split along y axis
            if i%2 == 0
                vy_min = bins_list[bin_id].v_split[2]
                vy_max = bins_list[bin_id].v_hi[2]
            else
                # lower vy values
                vy_min = bins_list[bin_id].v_lo[2]
                vy_max = bins_list[bin_id].v_split[2]
            end
                    
            if ((i==3) || (i==4) || (i==7) || (i==8))
                vx_min = bins_list[bin_id].v_split[1]
                vx_max = bins_list[bin_id].v_hi[1]
            else
                vx_min = bins_list[bin_id].v_lo[1]
                vx_max = bins_list[bin_id].v_split[1]
            end

                        
            
            if i==1
                istart = bins_list[bin_id].index_start
            else
                istart = bin_counts_acc_copy[i-1] + bins_list[bin_id].index_start
            end
            iend = bin_counts_acc_copy[i] + bins_list[bin_id].index_start - 1
            
            # split are zeros for now, we will compute them
            push!(bins_list, Bin(0.0, 0, [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],
                                 [vx_min, vy_min, vz_min], [vx_max, vy_max, vz_max], [0.0, 0.0, 0.0],
                                 istart, iend,
                                 bins_list[bin_id].level+1))

            compute_bin_weight_and_split!(particle_list, bins_list[n_bins+new_bins], split_type)
        end
    end
    
    bins_list[bin_id:n_bins + new_bins-1] = bins_list[bin_id+1:n_bins + new_bins]
    
    resize!(bins_list, n_bins + new_bins - 1)
#     bins_list = bins_list[1:3]
    # shift bins to left to overwrite bin
    return n_bins + new_bins - 1
end

refine_bin! (generic function with 1 method)

In [15]:
function init_bin(particle_list, bin_init, split_type)
    # create initial bin and set initial velocity splitting values
    
    vx_min = 3.1e8
    vx_max = -3.1e8
    vy_min = 3.1e8
    vy_max = -3.1e8
    vz_min = 3.1e8
    vz_max = -3.1e8
    
    if bin_init == LightSpeed
        print("initial bounds set to +- 1.03c\n")
        
    else
        print("initial bounds set to min-max velocities\n")
        
        for particle in particle_list
            if particle.v[1] < vx_min
                vx_min = particle.v[1]
            elseif particle.v[1] > vx_max
                vx_max = particle.v[1]
            end

            if particle.v[2] < vy_min
                vy_min = particle.v[2]
            elseif particle.v[2] > vy_max
                vy_max = particle.v[2]
            end

            if particle.v[3] < vz_min
                vz_min = particle.v[3]
            elseif particle.v[3] > vz_max
                vz_max = particle.v[3]
            end
        end
    end
    
                
    # compute stuff!!! 
    bin = Bin(0.0, 0, [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],
              [vx_min, vy_min, vz_min], [vx_max, vy_max, vz_max], [0.0, 0.0, 0.0],
              1, length(particle_list), 0)
    compute_bin_weight_and_split!(particle_list, bin, split_type)
    bin.v_split = [0.0, 0.0, 0.0]
    return bin
end

init_bin (generic function with 1 method)

In [16]:
function octree_merge!(particle_list, bin_list, target_particle_count, max_refinement, aggressiveness, split_type, bin_refinement)
    if (length(particle_list) > target_particle_count)
        do_refinement = true
        target_weight = bin_list[1].w / target_particle_count
        
        # experiment: split heaviest cell vs split cell based on octree criteria
        while (do_refinement)
            curr_num_particles = length(bin_list)*2
            if (curr_num_particles >= target_particle_count)
                do_refinement = false
            else
                maxweight = 0.0
                refine_bin_id = -1
                    
                if (bin_refinement == Greedy)
                # based on paper, attempt to refine bins greedily
                    for (i, bin) in enumerate(bin_list)
                        # check that we haven't reached maximum recursion level
                        if (bin.level < max_refinement)
                            if ((bin.nparticles > 2) && (bin.w > aggressiveness * target_weight))
                                refine_bin_id = i
                                break
                            end
                        end
                    end
                elseif (bin_refinement == Heaviest)
                # refine heaviest bin first
                    for (i, bin) in enumerate(bin_list)
                        # check that we haven't reached maximum recursion level
                        if ((bin.level < max_refinement) && (bin.w > maxweight))
                            if ((bin.nparticles > 2) && (bin.w > aggressiveness * maxweight))
                                refine_bin_id = i
                                maxweight = bin.w
                            end
                        end
                    end
                else
                # refine bin with heaviest average weight
                    for (i, bin) in enumerate(bin_list)
                        # check that we haven't reached maximum recursion level
                        if ((bin.level < max_refinement) && (bin.nparticles > 2) && (bin.w / bin.nparticles > maxweight))
                            if ((bin.nparticles > 2) && (bin.w > aggressiveness * maxweight))
                                refine_bin_id = i
                                maxweight = bin.w / bin.nparticles
                            end
                        end
                    end
                end
                if (refine_bin_id == -1)
                    print("No bins found to refine")
                    do_refinement = false
                else
                    refine_bin!(particle_list, bin_list, refine_bin_id, length(bin_list), split_type)
                end        
            end
        end
        # compute new particle list
    end
        
    for bin in bin_list
        compute_bin_values!(particle_list, bin)
    end
end

octree_merge! (generic function with 1 method)

In [17]:
function create_new_particles(bin_list)
    new_particles = []
    
#     new_particles = []
    for bin in bin_list
        if (bin.w > 0)
            v_sign = sign.(0.5 .- rand(3))
            
            push!(new_particles, Particle(bin.w / 2, (bin.v .+ (v_sign .* bin.c)), 1))
            push!(new_particles, Particle(bin.w / 2, (bin.v .- (v_sign .* bin.c)), 1))
        end
    end
    
    return new_particles
end

create_new_particles (generic function with 1 method)

In [18]:
function compute_stats(particle_list, moment_nos)
    ndens = 0.0
    vel = [0.0, 0.0, 0.0]
    
    for particle in particle_list
        ndens += particle.w
        vel += particle.w .* particle.v
    end
    
    vel = vel ./ ndens
    
    print("number density: ", ndens, "\n")
    print("velocity: ", vel, "\n")
    
    moments = fill(0.0, (length(moment_nos), ))
#     moments = Array{Float64}(0.0, length(moment_nos))
    
    energy = 0.0
     
    for particle in particle_list
        
        energy += particle.w * sum((particle.v .- vel) .^ 2)
    end
    println("Energy: ", energy / ndens)
    
    for particle in particle_list
        
        for (i, moment_no) in enumerate(moment_nos)
            moments[i] += particle.w * sum((particle.v .- vel) .^ moment_no)
        end
    end
    
    for (i, moment_no) in enumerate(moment_nos)
        print("moment no.", moment_no, ": ", moments[i] / ndens, "\n")
    end
end

compute_stats (generic function with 1 method)

In [19]:
function compute_bias!(particle_list, particle_list_new, moment_nos, bias_array, n_run)
    ndens = 0.0
    vel = [0.0, 0.0, 0.0]
    
    for particle in particle_list
        ndens += particle.w
        vel += particle.w .* particle.v
    end
    
    vel = vel ./ ndens
    
    moments = fill(0.0, (length(moment_nos), ))
    moments_new = fill(0.0, (length(moment_nos), ))
    
    for particle in particle_list
        for (i, moment_no) in enumerate(moment_nos)
            moments[i] += particle.w * sum((particle.v .- vel) .^ moment_no)
        end
    end
    
    for particle in particle_list_new
        for (i, moment_no) in enumerate(moment_nos)
            moments_new[i] += particle.w * sum((particle.v .- vel) .^ moment_no)
        end
    end
    
    for (i, moment_no) in enumerate(moment_nos)
        bias_array[n_run, i] += (moments_new[i] - moments[i]) / moments[i]
    end
end

compute_bias! (generic function with 1 method)

In [20]:
function create_particles(mass, T, n_particles)
    particle_list = Array{Particle}(undef, n_particles)
    
    for i=1:n_particles
        particle_list[i] = Particle(1.0, maxwellian(T, ar_mass), 0)
    end
    return particle_list
end

create_particles (generic function with 1 method)

In [21]:
function create_particles_bimodal(mass, T, n_particles, offset)
    particle_list = Array{Particle}(undef, n_particles * 2)
    
    for i=1:n_particles
        particle_list[i] = Particle(1.0, maxwellian(T, ar_mass) + offset, 0)
    end
    for i=n_particles+1:n_particles*2
        particle_list[i] = Particle(1.0, maxwellian(T, ar_mass) - offset, 0)
    end
    return particle_list
end

create_particles_bimodal (generic function with 1 method)

In [22]:
ar_mass = 6.633521356992e-26
T = 300.0
n_particles = 100000

n_runs = 10
moment_nos = [2, 4,6,8,10]

target_np = 1500
aggr = 1.0

inittype = MinMaxVel
# inittype = LightSpeed
splittype = MiddleSplit
# splittype = MeanVelSplit
refinement = Greedy
# refinement = Heaviest
refinment = BiggestAvg

# inittype = MinMaxVel
# splittype = MeanVelSplit
# refinement = Heaviest

v_ref = sqrt(2 * constants.k * T / ar_mass)
N_cell = 16
max_vel_to_v_ref = 5.5

5.5

In [23]:
bias_arr = fill(0.0, (n_runs, length(moment_nos)));

avg_np = 0

for run=1:n_runs
    
#     particle_list = create_particles(ar_mass, T, n_particles);
    particle_list = create_particles_bimodal(ar_mass, T, n_particles, [2 * v_ref, 0.0, 0.0]);
    @time binlist_test = cell_merge!(particle_list, N_cell,
                                     [-v_ref * max_vel_to_v_ref, -v_ref * max_vel_to_v_ref, -v_ref * max_vel_to_v_ref],
                                     [v_ref * max_vel_to_v_ref, v_ref * max_vel_to_v_ref, v_ref * max_vel_to_v_ref]);
    
    newparticles = create_new_particles(binlist_test);
    avg_np += length(newparticles)
    println("post-merge:", length(newparticles))
    compute_bias!(particle_list, newparticles, moment_nos, bias_arr, run);
end

println(avg_np / n_runs)

  0.583207 seconds (2.17 M allocations: 162.436 MiB, 22.62% gc time)
post-merge:1454
  0.284187 seconds (1.23 M allocations: 115.751 MiB, 37.35% gc time)
post-merge:1478
  0.350881 seconds (1.23 M allocations: 115.748 MiB, 43.51% gc time)
post-merge:1460
  0.245179 seconds (1.23 M allocations: 115.744 MiB, 22.66% gc time)
post-merge:1432
  0.223885 seconds (1.23 M allocations: 115.747 MiB, 24.77% gc time)
post-merge:1454
  0.222957 seconds (1.23 M allocations: 115.745 MiB, 22.99% gc time)
post-merge:1440
  0.352024 seconds (1.23 M allocations: 115.747 MiB, 48.28% gc time)
post-merge:1450
  0.318365 seconds (1.23 M allocations: 115.746 MiB, 25.39% gc time)
post-merge:1444
  0.235804 seconds (1.23 M allocations: 115.741 MiB, 25.76% gc time)
post-merge:1416
  0.240190 seconds (1.23 M allocations: 115.748 MiB, 22.06% gc time)
post-merge:1460
1448.8


In [137]:
mean(bias_arr, dims=(1))

1×4 Array{Float64,2}:
 -0.000660577  -0.00224959  -0.00584906  -0.0115643

In [134]:
std(bias_arr, dims=(1))

1×4 Array{Float64,2}:
 3.76183e-5  0.000175562  0.000528134  0.00125126

In [147]:
bias_arr = fill(0.0, (n_runs, length(moment_nos)));
avg_np = 0
for run=1:n_runs
#     
#     particle_list = create_particles(ar_mass, T, n_particles);
    particle_list = create_particles_bimodal(ar_mass, T, n_particles, [2 * v_ref, 0.0, 0.0]);
    binlist_test = [init_bin(particle_list, inittype, splittype)];
    @time octree_merge!(particle_list, binlist_test, target_np, 5, aggr, splittype, refinement);
    newparticles = create_new_particles(binlist_test);
    println("post-merge:", length(newparticles))
    avg_np += length(newparticles)
    compute_bias!(particle_list, newparticles, moment_nos, bias_arr, run);
    
end

println(avg_np / n_runs)

initial bounds set to min-max velocities
No bins found to refine  0.510436 seconds (3.20 M allocations: 364.390 MiB, 12.07% gc time)
post-merge:1200
initial bounds set to min-max velocities
No bins found to refine  0.502298 seconds (3.20 M allocations: 364.246 MiB, 14.92% gc time)
post-merge:1182
initial bounds set to min-max velocities
No bins found to refine  0.557702 seconds (3.20 M allocations: 364.347 MiB, 26.61% gc time)
post-merge:1194
initial bounds set to min-max velocities
No bins found to refine  0.700425 seconds (3.20 M allocations: 363.972 MiB, 16.24% gc time)
post-merge:1184
initial bounds set to min-max velocities
No bins found to refine  0.589259 seconds (3.20 M allocations: 364.244 MiB, 11.62% gc time)
post-merge:1226
initial bounds set to min-max velocities
No bins found to refine  0.611914 seconds (3.20 M allocations: 364.211 MiB, 14.29% gc time)
post-merge:1182
initial bounds set to min-max velocities
No bins found to refine  0.762455 seconds (3.20 M allocations: 36

In [148]:
mean(bias_arr, dims=(1))

1×4 Array{Float64,2}:
 -0.0156434  -0.0861325  -0.219666  -0.382473

In [149]:
std(bias_arr, dims=(1))

1×4 Array{Float64,2}:
 0.000479224  0.00192157  0.00545399  0.0102239

In [ ]:
# middle split:  -0.12505  -0.27281  -0.377198  -0.465822
# meanvel split:  -0.0913615  -0.338818  -0.61502  -0.81325
# meanvel + heaviest:   -0.106529  -0.379313  -0.661266  -0.847062
# middle split + heaviest:  -0.130482  -0.293857  -0.427118  -0.556371